In [ ]:
---
layout: post
title:  Complete Pong Game Code Implementation
description: Complete HTML, CSS, and JavaScript code for building a fully functional 2-player Pong game
categories: ['Game Development', 'JavaScript', 'Canvas API', 'Code Implementation']
permalink: /custompong
menu: nav/tools_setup.html
toc: True
comments: True
---

## 🎮 Pong Game Demo

<div class="game-canvas-container" style="text-align:center;">
  <canvas id="pongCanvas" width="800" height="500"></canvas>
  <br>
  <button id="restartBtn">Restart Game</button>
</div>

<style>
  .game-canvas-container {
    margin-top: 20px;
  }
  #pongCanvas {
    border: 2px solid #fff;
    background: #000;
  }
  #restartBtn {
    display: none;
    margin-top: 15px;
    padding: 10px 20px;
    font-size: 18px;
    border: none;
    border-radius: 6px;
    background: #4caf50;
    color: white;
    cursor: pointer;
  }
  #restartBtn:hover {
    background: #45a049;
  }
</style>

<script>
// =============================
// Pong (Object-Oriented Version)
// =============================
// This refactor makes the game easy to modify and includes student TODOs.
// Teachers: Encourage students to change the Config values and complete TODOs.

// -----------------------------
// Config: Tweak these values
// -----------------------------
const Config = {
  canvas: { width: 800, height: 500 },
  paddle: { imagePath: "/images/pingpong/happymort.png", width: 20, height: 120, speed:7 },
  ball: { radius: 10, baseSpeedX: 5, maxRandomY: 2, spinFactor: 0.3 },
  rules: { winningScore: 10 },
  keys: {
    // TODO[Students]: Remap keys if desired
    p1Up: "w", p1Down: "s",
    p2Up: "i", p2Down: "k"
  },
  visuals: { bg: "#000", fg: "#fff", text: "#fff", gameOver: "red", win: "green" }
};

// Basic vector helper for clarity
class Vector2 {
  constructor(x = 0, y = 0) { this.x = x; this.y = y; }
}

class Paddle {
  constructor(x, y, width, height, speed, boundsHeight, imagePath) {
    this.position = new Vector2(x, y);
    this.width = width;
    this.height = height;
    this.speed = speed;
    this.boundsHeight = boundsHeight;
    this.imagePath = imagePath
  }
  move(dy) {
    this.position.y = Math.min(
      this.boundsHeight - this.height,
      Math.max(0, this.position.y + dy)
    );
  }
  rect() { return { x: this.position.x, y: this.position.y, w: this.width, h: this.height }; }
}

class Ball {
  constructor(radius, boundsWidth, boundsHeight) {
    this.radius = radius;
    this.boundsWidth = boundsWidth;
    this.boundsHeight = boundsHeight;
    this.position = new Vector2();
    this.velocity = new Vector2();
    this.reset(true);
  }
  reset(randomDirection = false) {
    this.position.x = this.boundsWidth / 2;
    this.position.y = this.boundsHeight / 2;
    const dir = randomDirection && Math.random() > 0.5 ? 1 : -1;
    this.velocity.x = dir * Config.ball.baseSpeedX;
    this.velocity.y = (Math.random() * (2 * Config.ball.maxRandomY)) - Config.ball.maxRandomY;
  }
  update() {
    this.position.x += this.velocity.x;
    this.position.y += this.velocity.y;
    // Bounce off top/bottom walls
    if (this.position.y + this.radius > this.boundsHeight || this.position.y - this.radius < 0) {
      this.velocity.y *= -1;
    }
  }
}

class Input {
  constructor() {
    this.keys = {};
    document.addEventListener("keydown", e => this.keys[e.key] = true);
    document.addEventListener("keyup", e => this.keys[e.key] = false);
  }
  isDown(k) { return !!this.keys[k]; }
}

class Renderer {
  constructor(ctx) { this.ctx = ctx; }
  clear(w, h) {
    this.ctx.fillStyle = Config.visuals.bg;
    this.ctx.fillRect(0, 0, w, h);
  }
  rect(r, color = Config.visuals.fg) {
    this.ctx.fillStyle = color;
    this.ctx.fillRect(r.x, r.y, r.w, r.h);
  }
  circle(ball, color = Config.visuals.fg) {
    this.ctx.fillStyle = color;
    this.ctx.beginPath();
    this.ctx.arc(ball.position.x, ball.position.y, ball.radius, 0, Math.PI * 2);
    this.ctx.closePath();
    this.ctx.fill();
  }
  text(t, x, y, color = Config.visuals.text) {
    this.ctx.fillStyle = color;
    this.ctx.font = "30px Arial";
    this.ctx.fillText(t, x, y);
  }
}

class Game {
  constructor(canvasEl, restartBtn) {
    // Canvas
    this.canvas = canvasEl;
    this.ctx = canvasEl.getContext('2d');
    this.renderer = new Renderer(this.ctx);

    // Systems
    this.input = new Input();

    // Entities
    const { imagePath, width, height, speed, imagePath} = Config.paddle;
    this.paddleLeft = new Paddle(0, (Config.canvas.height - height) / 2, width, height, speed, Config.canvas.height, imagePath);
    this.paddleRight = new Paddle(Config.canvas.width - width, (Config.canvas.height - height) / 2, width, height, speed, Config.canvas.height, imagePath);
    this.ball = new Ball(Config.ball.radius, Config.canvas.width, Config.canvas.height);

    // Rules/state
    this.scores = { p1: 0, p2: 0 };
    this.gameOver = false;
    this.restartBtn = restartBtn;
    this.restartBtn.addEventListener("click", () => this.restart());

    this.loop = this.loop.bind(this);
  }

  // -------------------------------
  // TODO[Students]: Add an AI player
  // Replace the right paddle control with simple AI:
  // if (this.ball.position.y > centerY) move down else move up.
  // Try making difficulty adjustable with a speed multiplier.
  // -------------------------------

  handleInput() {
    if (this.gameOver) return;
    // Player 1 controls
    if (this.input.isDown(Config.keys.p1Up)) this.paddleLeft.move(-this.paddleLeft.speed);
    if (this.input.isDown(Config.keys.p1Down)) this.paddleLeft.move(this.paddleLeft.speed);
    // Player 2 controls (human). Swap to AI per TODO above.
    if (this.input.isDown(Config.keys.p2Up)) this.paddleRight.move(-this.paddleRight.speed);
    if (this.input.isDown(Config.keys.p2Down)) this.paddleRight.move(this.paddleRight.speed);
  }

  update() {
    if (this.gameOver) return;
    this.ball.update();

    // Paddle collisions with ball
    const hitLeft = this.ball.position.x - this.ball.radius < this.paddleLeft.width &&
      this.ball.position.y > this.paddleLeft.position.y &&
      this.ball.position.y < this.paddleLeft.position.y + this.paddleLeft.height;

    if (hitLeft) {
      this.ball.velocity.x *= -1;
      const delta = this.ball.position.y - (this.paddleLeft.position.y + this.paddleLeft.height / 2);
      this.ball.velocity.y = delta * Config.ball.spinFactor; // "spin"
    }

    const hitRight = this.ball.position.x + this.ball.radius > (Config.canvas.width - this.paddleRight.width) &&
      this.ball.position.y > this.paddleRight.position.y &&
      this.ball.position.y < this.paddleRight.position.y + this.paddleRight.height;

    if (hitRight) {
      this.ball.velocity.x *= -1;
      const delta = this.ball.position.y - (this.paddleRight.position.y + this.paddleRight.height / 2);
      this.ball.velocity.y = delta * Config.ball.spinFactor;
    }

    // Scoring
    if (this.ball.position.x - this.ball.radius < 0) {
      this.scores.p2++;
      this.checkWin() || this.ball.reset();
    } else if (this.ball.position.x + this.ball.radius > Config.canvas.width) {
      this.scores.p1++;
      this.checkWin() || this.ball.reset();
    }
  }

  checkWin() {
    if (this.scores.p1 >= Config.rules.winningScore || this.scores.p2 >= Config.rules.winningScore) {
      this.gameOver = true;
      this.restartBtn.style.display = "inline-block";
      return true;
    }
    return false;
  }

  draw() {
    this.renderer.clear(Config.canvas.width, Config.canvas.height);
    this.renderer.rect(this.paddleLeft.rect());
    this.renderer.rect(this.paddleRight.rect());
    this.renderer.circle(this.ball);
    this.renderer.text(this.scores.p1, Config.canvas.width / 4, 50);
    this.renderer.text(this.scores.p2, 3 * Config.canvas.width / 4, 50);
    if (this.gameOver) {
      this.renderer.text("Game Over", Config.canvas.width / 2 - 80, Config.canvas.height / 2 - 20, Config.visuals.gameOver);
      const msg = this.scores.p1 >= Config.rules.winningScore ? "Player 1 Wins!" : "Player 2 Wins!";
      this.renderer.text(msg, Config.canvas.width / 2 - 120, Config.canvas.height / 2 + 20, Config.visuals.win);
    }
  }

  restart() {
    this.scores.p1 = 0; this.scores.p2 = 0;
    this.paddleLeft.position.y = (Config.canvas.height - this.paddleLeft.height) / 2;
    this.paddleRight.position.y = (Config.canvas.height - this.paddleRight.height) / 2;
    this.ball.reset(true);
    this.gameOver = false;
    this.restartBtn.style.display = "none";
  }

  loop() {
    this.handleInput();
    this.update();
    this.draw();
    requestAnimationFrame(this.loop);
  }
}

// -------------------------------
// Bootstrapping
// -------------------------------
const canvas = document.getElementById('pongCanvas');
const restartBtn = document.getElementById('restartBtn');

// Ensure canvas matches Config every load (keeps HTML in sync)
canvas.width = Config.canvas.width;
canvas.height = Config.canvas.height;

const game = new Game(canvas, restartBtn);
game.loop();

// -----------------------------------------
// Student Challenges (inline TODO checklist)
// -----------------------------------------
// 1) Make it YOUR game: change colors in Config.visuals, dimensions/speeds in Config.
// 2) Add AI: implement simple tracking for right paddle in handleInput (hint above).
// 3) Rally speed-up: every time the ball hits a paddle, slightly increase |velocity.x|.
// 4) Center line + score SFX: draw a dashed midline; play an audio on score.
// 5) Power-ups: occasionally spawn a small rectangle; when ball hits it, apply effect (bigger paddle? faster ball?).
// 6) Pause/Resume: map a key to toggle pause state in Game and skip updates when paused.
// 7) Win screen polish: show a replay countdown, then auto-restart.


SyntaxError: invalid character '🏓' (U+1F3D3) (3908889265.py, line 3)